# 写在前面

做这一个系列的notebooks，是为了帮助自己好好梳理一下自然语言处理（NLP）过程中的方法和模型。

一共分为以下几个部分：
+ 数据预处理
+ 词袋模型的使用
+ n-gram及共现的概念
+ 词的分布式表示
+ torchtext的使用
+ CNN与RNN【RNN暂缺】
+ Attention机制

整个notebook系列使用的数据是星际迷航原初（TOS）剧集中的所有台词，参见raw_data文件夹。

<img src='images/startrek.jpg'>

# 1 - 数据预处理

文本数据预处理中有几个核心概念：
+ **tokenize**：分词，将句子切分成一个个单词/词汇与符号
+ **filter**：过滤，利用停词表（stopwords）将一些非常常见但是没有意义的词去掉，如is/the/to...
+ **POS tagging**：词性标注，将词划分为名次/动词/形容词...
+ **stemming/lemmatisation**：词干提取/词形还原，将词还原成最原始的状态，如stopped->stop

以上所说的方法并非预处理步骤中所必须的，应该根据具体问题来进行选择。

下面会展示[**nltk**](http://www.nltk.org/)和[**jieba**](https://github.com/fxsjy/jieba)中各个方法的具体使用。

In [3]:
import nltk
from nltk.corpus import stopwords 

import jieba
import jieba.analyse

## 分词

分词函数可以自己定义，也可以使用现成的->很多NLP的库都会提供tokenizer或相应机制。

In [15]:
# 自定义
def tokenize(text):
    text = text.lower() #将字符串全部降为小写
    text = re.split(r',|\.|\?|!|"|\'| |\n',text)
    return text


# nltk
en = "Maybe we can’t stroll to the music of the lute. We must march to the sound of drums."
en_sents = nltk.sent_tokenize(en) #利用“./?/!”对文本段进行分句
en_words = nltk.word_tokenize(en) #利用所有的标点符号对文本进行分词

# jieba
ch = "选择洗衣机、汽车、镭射音响，还有电动开罐器。"
ch_words = list(jieba.cut(ch))

print("en_sents:",en_sents)
print("en_words:",en_words)
print("ch_words:",ch_words)

en_sents: ['Maybe we can’t stroll to the music of the lute.', 'We must march to the sound of drums.']
en_words: ['Maybe', 'we', 'can', '’', 't', 'stroll', 'to', 'the', 'music', 'of', 'the', 'lute', '.', 'We', 'must', 'march', 'to', 'the', 'sound', 'of', 'drums', '.']
ch_words: ['选择', '洗衣机', '、', '汽车', '、', '镭射', '音响', '，', '还有', '电动', '开罐器', '。']


由于中文的分词较为复杂，可以加载自定义的词典来绑定词组，使用`jieba.load_userdict(file_name)`将准备好的txt格式词典载入，每行一个词语。

## 过滤

In [8]:
# nltk
# from nltk.corpus import stopwords
stopwords = stopwords.words('english') #得到的是一个list
print("en_stopwords:",stopwords)

en_stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don',

可以看到nltk提供的停词表词汇量其实很少，往往无法满足深度过滤的需求。可以用append自行添加词汇。

而jieba则没有提供stopwords，需要自己定义，source文件夹中包含一份中文停词stopwords.txt。

In [9]:
filtered_words = []
for word in en_words:
    if word not in stopwords:
        filtered_words.append(word)
print(filtered_words)

['Maybe', '’', 'stroll', 'music', 'lute', '.', 'We', 'must', 'march', 'sound', 'drums', '.']


## 词性标注

有时候我们只关注名词、动词或形容词（比如在提取评论关键词时），这就需要对词的词性进行区分。

In [17]:
# nltk
for word,pos in nltk.pos_tag(filtered_words): #传参为list
    if pos in ('NN','NNS','VB'):
        print(word,pos)
print()
# jieba
for word,pos in jieba.posseg.cut(ch): #传参为str
    if pos in ('v','n'):
        print(word,pos)

stroll NN
music NN
lute NN
march VB
sound NN
drums NNS

选择 v
洗衣机 n
汽车 n
音响 n
还有 v
电动 n
开罐器 n


## 词型还原/词干提取

这两个概念仅涉及西方语系语种，和中文不搭边。nltk中两种处理手段的工具都有提供。

从体验上来看，`词干提取`风险较大，经常让单词缺胳膊少腿，对不规则变化也无法识别。而`词性还原`本质是基于字典，虽然准确度可以有所保证，但加载比较慢，而且还需要指定词性。

总的来说，使用起来都不太理想。

In [27]:
stemmer = nltk.PorterStemmer()
lemmatizer = nltk.WordNetLemmatizer()

print('hiking:',stemmer.stem('hiking'),lemmatizer.lemmatize('hiking','v'))
print('ran:',stemmer.stem('ran'),lemmatizer.lemmatize('ran','v'))

hiking: hike hike
ran: ran run


## SpaCy
基于以上的种种不方便，在此提供另一个库[SpaCy](https://spacy.io/)。它用一个模型解决所有分词、词型还原、词性标注、停词信息等问题。

（不过随着文本长度增加，其处理速度会显著变慢）

In [34]:
import spacy

nlp = spacy.load("en_core_web_sm") #需下载
doc = nlp(en)

for token in doc[:4]:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Maybe maybe ADV RB advmod Xxxxx True False
we -PRON- PRON PRP nsubj xx True True
ca can VERB MD aux xx True True
n’t not PART RB neg x’x False True
